# 1. Fetching Data

In [88]:

import pandas as pd
import requests
from bs4 import BeautifulSoup


In [97]:
raw_data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

parsed =  BeautifulSoup(raw_data.text, 'lxml')
tables = parsed.find_all('table')
table = tables[0]
subtags = table.find_all('tr') #Find all the <tr> Tags and fetch into array
initTag = subtags[3]  #First 3 rows are unassigned and hence ignoring them
subtags = subtags[4:] 

# 2. Creating and Building Dataframe

In [91]:
column_names = [ 'PostalCode','Borough', 'Neighborhood']
df = pd.DataFrame(columns= column_names)


In [92]:
#A function that takes input an array of <td> and returns the code, borough and neighbourhood as per the required format
def getValues(values):
    if(True):
        code = values[0].get_text()
        borough = values[1].get_text()
        neibr = values[2].get_text().rstrip()
        if(borough != 'Not assigned'):
            if(neibr != 'Not assigned'):
                return code, borough, neibr
            else:
                return code, borough, borough
        else:
            return None, None, None
    else:
        return None,None,None

#Iteratively traversing the list of <tr> tags and cumulating the data into a dataframe
code, bor, nei = getValues(initTag.find_all('td'))
for tag in subtags:
    next_code, next_bor, next_nei = getValues(tag.find_all('td'))
    if(next_bor==None):
        continue
    if(next_code==code):
        nei = nei + ', '+next_nei
    else:
        df = df.append({'PostalCode':code,'Borough':bor, 'Neighborhood':nei}, ignore_index=True)
        code = next_code
        bor = next_bor
        nei = next_nei

df = df.append({'PostalCode':code,'Borough':bor, 'Neighborhood':nei}, ignore_index=True)    
       

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


,PostalCode,Borough,Neighborhood
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing Centre969 Eastern
101,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."
102,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen..."


In [93]:
#Shape
df.shape

(103, 3)